In this notebook, we'll explore some statistical and machine learning libraries.

We'll be working with a data set consisting of a sample of 200 subjects who were part of a study on survival of patients following admission to an adult intensive care unit. The goal of the study was to develop a logistic regression model to predict the probability of survival to hospital discharge of these patients.

In [1]:
import pandas as pd

In [2]:
icu = pd.read_csv('../data/icu.csv')

In [3]:
icu

,ID,STA,AGE,SEX,RACE,SER,CAN,CRN,INF,CPR,...,HRA,PRE,TYP,FRA,PO2,PH,PCO,BIC,CRE,LOC
0,552,0,16,0,1,1,0,0,0,0,...,140,0,1,1,0,0,0,0,0,0
1,102,0,16,1,1,0,0,0,0,0,...,111,0,1,0,0,0,0,0,0,0
2,837,0,17,1,3,0,0,0,0,0,...,140,0,1,0,0,0,0,0,0,0
3,863,0,17,0,3,1,0,0,0,0,...,78,0,1,0,0,0,0,0,0,0
4,829,0,17,0,1,1,0,0,0,0,...,78,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,877,0,88,1,1,0,0,1,0,0,...,88,0,1,0,0,0,0,0,0,0
196,881,0,89,1,1,1,0,0,0,0,...,114,0,1,0,0,0,1,0,0,2
197,204,1,91,0,1,0,0,0,1,0,...,125,0,1,0,0,0,1,0,0,0
198,674,0,91,0,1,0,0,1,1,0,...,125,0,1,0,0,0,0,0,0,0


The variables are as follows:

|Variable | Description | Codes/Values|
|---|---|---|
| ID | Identification Code | ID Number|
| STA | Vital Status | 0 = Lived<br /> 1 = Died |
| AGE | Age | Years |
| SEX | Sex | 0 = Male<br /> 1 = Female | 
| RACE | Race | 1 = White<br />2 = Black<br />3 = Other |
| SER | Service at ICU Admission | 0 = Medical<br />1 = Surgical |
| CAN | Cancer Part of Present Problem | 0 = No<br />1 = Yes |
| CRN | History of Chronic Renal Failure | 0 = No<br />1 = Yes |
| INF | Infection Probable at ICU Admission | 0 = No<br />1 = Yes |
| CPR | CPR Prior to ICU Admission | 0 = No<br />1 = Yes |
| SYS | Systolic Blood Pressure at ICU Admission | mm Hg |
| HRA | Heart Rate at ICU Admission | Beats/min |
| PRE | Previous Admission to an ICU Within 6 Months | 0 = No<br />1 = Yes |
| TYP | Type of Admission | 0 = Elective<br />1 = Emergency |
| FRA | Long Bone, Multiple, Neck, Single Area, or Hip Fracture | 0 = No<br />1 = Yes |
| PO2 | PO2 from Initial Blood Gases | 0: $>$60<br />1: $\leq$ 60 |
| PH | PH from Initial Blood Gases | 0: $\geq$ 7.25<br />1: $<$7.25 |
| PCO | PCO2 from Initial Blood Gases | 0: $\leq$ 45<br />1: $>$45 |
| BIC | Bicarbonate from Initial Blood Gases | 0: $\geq$ 18<br />1: $<$ 18 |
| CRE | Creatinine from Initial Blood Gases | 0: $\leq$2.0<br />1: $>$2.0 |
| LOC | Level of Consciousness at ICU Admission | 0 = No Coma or Deep Stupor<br />1 = Deep Stupor<br />2 = Coma |

A useful library for conducting statistical tests is the _statsmodels_ library.

Let's say we want to test the null hypothesis that there is no difference in average age between those that die compared to those that do not die against the alternative hypothesis that there is a difference. 

For this, we can use a t-test.

In [4]:
from statsmodels.stats.weightstats import ttest_ind

In [5]:
ttest_ind(x1 = icu[icu['STA'] == 0]['AGE'],               # observations that do not die
          x2 = icu[icu['STA'] == 1]['AGE'],               # observations that die
          alternative = 'two-sided',                      # can perform a one-sided test by using 'larger' or 'smaller'
          usevar = 'unequal')                             # We'll Welch's t-test

(-3.067979086671082, 0.0030443793137557764, 71.40119955116975)

This function returns the test statistic, the p-value, and the degrees of freedom.

In this case, at the 0.05 significance level, we can reject the null hypothesis.

# Statistical Modeling Approach - _statsmodels_

If we want to build a logistic regression model, we can make use statsmodels along with the patsy library to build a design matrix.

In [6]:
from patsy import dmatrices
import statsmodels.api as sm

In [7]:
y, X = dmatrices('STA ~ AGE',                       # Target variable ~ Predictor variable(s)
                 icu,                               # Dataset
                 return_type = 'dataframe')

In [8]:
X

,Intercept,AGE
0,1.0,16.0
1,1.0,16.0
2,1.0,17.0
3,1.0,17.0
4,1.0,17.0
...,...,...
195,1.0,88.0
196,1.0,89.0
197,1.0,91.0
198,1.0,91.0


Now, we'll use the Logit class from statsmodels to build our model.

In [9]:
logit = sm.Logit(y, X)

Fit the model and save the result.

In [10]:
res = logit.fit()

Optimization terminated successfully.
         Current function value: 0.480766
         Iterations 6


We can see the parameters using the `params` attribute.

In [11]:
res.params

Intercept   -3.058513
AGE          0.027543
dtype: float64

And we can get a statistical summary using the `summary()` method.

In [12]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    STA   No. Observations:                  200
Model:                          Logit   Df Residuals:                      198
Method:                           MLE   Df Model:                            1
Date:                Sat, 10 Jul 2021   Pseudo R-squ.:                 0.03924
Time:                        15:51:14   Log-Likelihood:                -96.153
converged:                       True   LL-Null:                       -100.08
Covariance Type:            nonrobust   LLR p-value:                  0.005069
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.0585      0.696     -4.394      0.000      -4.423      -1.694
AGE            0.0275      0.011      2.607      0.009       0.007       0.048
==============================================================================
"""

If we want to include other factors, say race, we can do that by including them in our 

In [13]:
y, X = dmatrices('STA ~ AGE + SEX', 
                 icu, 
                 return_type = 'dataframe')

In [14]:
X

,Intercept,AGE,SEX
0,1.0,16.0,0.0
1,1.0,16.0,1.0
2,1.0,17.0,1.0
3,1.0,17.0,0.0
4,1.0,17.0,0.0
...,...,...,...
195,1.0,88.0,1.0
196,1.0,89.0,1.0
197,1.0,91.0,0.0
198,1.0,91.0,0.0


In [15]:
logit = sm.Logit(y, X)
res = logit.fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.480764
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    STA   No. Observations:                  200
Model:                          Logit   Df Residuals:                      197
Method:                           MLE   Df Model:                            2
Date:                Sat, 10 Jul 2021   Pseudo R-squ.:                 0.03925
Time:                        15:51:14   Log-Likelihood:                -96.153
converged:                       True   LL-Null:                       -100.08
Covariance Type:            nonrobust   LLR p-value:                   0.01969
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.0567      0.699     -4.374      0.000      -4.426      -1.687
AGE            0.0276      0.011      2.589      0.010       0.007       0.048
SEX           -0.0113      0.372     -0.030      0.976      -0.740       0.717
==============================================================================
"""

If we want to include categorical variables with more than 2 levels, we can encode them using `C()`.

In [16]:
y, X = dmatrices('STA ~ AGE + SEX + C(RACE)', icu, return_type = 'dataframe')

In [17]:
X

,Intercept,C(RACE)[T.2],C(RACE)[T.3],AGE,SEX
0,1.0,0.0,0.0,16.0,0.0
1,1.0,0.0,0.0,16.0,1.0
2,1.0,0.0,1.0,17.0,1.0
3,1.0,0.0,1.0,17.0,0.0
4,1.0,0.0,0.0,17.0,0.0
...,...,...,...,...,...
195,1.0,0.0,0.0,88.0,1.0
196,1.0,0.0,0.0,89.0,1.0
197,1.0,0.0,0.0,91.0,0.0
198,1.0,0.0,0.0,91.0,0.0


In [18]:
logit = sm.Logit(y, X)
res = logit.fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.477623
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    STA   No. Observations:                  200
Model:                          Logit   Df Residuals:                      195
Method:                           MLE   Df Model:                            4
Date:                Sat, 10 Jul 2021   Pseudo R-squ.:                 0.04552
Time:                        15:51:14   Log-Likelihood:                -95.525
converged:                       True   LL-Null:                       -100.08
Covariance Type:            nonrobust   LLR p-value:                   0.05837
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.9216      0.715     -4.087      0.000      -4.323      -1.521
C(RACE)[T.2]    -1.0054      1.065     -0.944      0.345      -3.093       1.082
C(RACE)[T.3]     0.2154      0.837      0.257      0.797      -1.424       1.855
AGE              0.0260      0.011      2.428      0.015       0.005       0.047
SEX             -0.0125      0.372     -0.033      0.973      -0.742       0.717
================================================================================
"""

# Machine Learning Approach - _scikit-learn_

We can also build a logistic regression function using the _scikit-learn_ library.

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
X = icu[['AGE']]              # Predictor(s)
y = icu['STA']                # Targer

In [21]:
logreg = LogisticRegression()

In [22]:
logreg.fit(X, y)

LogisticRegression()

If we want to inspect the output, we can access the `coef_` and `intercept_` attributes.

In [23]:
logreg.coef_

array([[0.02753969]])

In [24]:
logreg.intercept_

array([-3.05832222])

If you compare these with the result from the statsmodels version, you'll notice that they are slightly different. This is because by default, scikit-learn includes **reguarlization** in its logistic regression model by default, which will shrink the coefficients towards zero.

What if we want to include categorical variables?

In [25]:
X = icu[['AGE', 'SEX', 'RACE']]
y = icu['STA']

We can encode categorical variables using the `get_dummies()` function from pandas.

In [26]:
pd.get_dummies(X, columns = ['RACE'], drop_first = True)

,AGE,SEX,RACE_2,RACE_3
0,16,0,0,0
1,16,1,0,0
2,17,1,0,1
3,17,0,0,1
4,17,0,0,0
...,...,...,...,...
195,88,1,0,0
196,89,1,0,0
197,91,0,0,0
198,91,0,0,0


In [27]:
X = pd.get_dummies(X, columns = ['RACE'], drop_first = True)

In [28]:
logreg = LogisticRegression()
logreg.fit(X, y)

LogisticRegression()

In [29]:
coefficients = pd.DataFrame({'variable': X.columns, 'coefficient': logreg.coef_[0]})
coefficients

,variable,coefficient
0,AGE,0.026540
1,SEX,-0.010914
2,RACE_2,-0.538762
3,RACE_3,0.155648


A more typical machine learning workflow would be to include more of the variables and let the algorithm determine which have predictive power.

Specifically, we'll use a **random forest** algorithm.

This is a very flexible model and consequently will perform well on the data that it is trained on. To get a fair assessment of how well our model makes predictions, we'll set aside some our our data as a **test set**.

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [31]:
X = icu.drop(columns = ['ID', 'STA'])
y = icu['STA']

In [32]:
categorical_variables = ['RACE', 'LOC']
X = pd.get_dummies(X, columns = categorical_variables)

Split the data into a training set and a test set.

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify = y,
                                                    test_size = 0.25,
                                                    random_state = 321)

In [34]:
clf = RandomForestClassifier(random_state = 321)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=321)

Now, we can see how well it performs on our test set.

In [35]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

First, generate predictions.

In [36]:
y_pred = clf.predict(X_test)

In [37]:
accuracy_score(y_test, y_pred)

0.88

In [38]:
confusion_matrix(y_test, y_pred)

array([[39,  1],
       [ 5,  5]])

In [39]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.97      0.93        40
           1       0.83      0.50      0.62        10

    accuracy                           0.88        50
   macro avg       0.86      0.74      0.78        50
weighted avg       0.88      0.88      0.87        50



We can also look at the predicted probabilities.

In [40]:
y_proba = clf.predict_proba(X_test)[:,1]

In [41]:
roc_auc_score(y_test, y_proba)

0.85125

One nice feature of a random forest model is that it will tell you which variables it relies most on to make predictions.

In [42]:
importances = pd.DataFrame({'variable': X.columns, 'importance': clf.feature_importances_})
importances.sort_values('importance', ascending = False)

,variable,importance
0,AGE,0.175077
7,SYS,0.173841
8,HRA,0.141376
20,LOC_0,0.098737
21,LOC_1,0.043381
22,LOC_2,0.042483
4,CRN,0.035584
1,SEX,0.033736
6,CPR,0.029379
2,SER,0.028714
